In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../flaming-choripan') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [34]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

rootdir = '../save/alerceZTFv7.1'
method = 'mcmc'
cs.get_all_incorrects_fittings(rootdir, method)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[]

In [35]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

method = 'mcmc'
rank, rank_bdict, band_names = cs.get_ranks(rootdir, method)
print(rank)
for b in band_names:
    print(rank_bdict[b])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mb-rank:
(1) - ZTF19abclykm: 1573.4415531111683
(2) - ZTF19aafljiq: 439.10263433567843
(3) - ZTF20aawmlfq: 260.0581272806624
(4) - ZTF19aaxfzao: 150.1000051622084
(5) - ZTF19aamhhae: 105.19817253719822
(6) - ZTF20aayprqz: 94.49655621888238
(7) - ZTF18abshezu: 88.40634320667245
(8) - ZTF18aavjcpf: 80.64340674204233
(9) - ZTF19abpbopt: 69.71207033939248
(10) - ZTF19aaeopgw: 68.17809256420074
g-rank:
(1) - ZTF19abclykm: 1022.6873097632855
(2) - ZTF19aafljiq: 511.7394954111892
(3) - ZTF20aawmlfq: 333.5915861291833
(4) - ZTF19aaxfzao: 117.29049121013826
(5) - ZTF18abshezu: 113.60690788116085
(6) - ZTF19aaeopgw: 112.19445262879353
(7) - ZTF19aamhhae: 98.97844611571261
(8) - ZTF20aayprqz: 95.16629529709608
(9) - ZTF19abpbopt: 73.75058619729101
(10) - ZTF19aaumvig: 66.84375350833443
r-rank:
(1) - ZTF19abclykm: 2124.195796459051
(2) - ZTF19aafljiq: 366.4657732601677
(3) - ZTF20aawmlfq: 186.5246684321415
(4) 

In [11]:
%load_ext autoreload
%autoreload 2
from synthsne.generators import curve_statistics as cs

curve_statistics
    
methods = ['curve_fit', 'mcmc']
band_names = ['g', 'r']
rootdir = '/home/opimentel/tesis/astro-lightcurves-handler/save/alerceZTFv5.1'

results_dict = {}
rank = {}
rank_bdict = {}
for method in methods:
    results_dict[method] = {
        'mb-fit-error':[],
        'mb-null-fits':0,
        'mb-n':0,
    }
    for b in band_names:
        results_dict[method].update({
            f'{b}-fit-error':[],
            f'{b}-null-fits':0,
            f'{b}-n':0,
        })

    rank[method] = TopRank('mb-rank')
    rank_bdict[method] = {b:TopRank(f'{b}-rank') for b in band_names}

### loop
for method in methods:
    load_rootdir = f'{rootdir}/{method}'
    filedirs = get_filedirs(load_rootdir, fext='ferror')
    for filedir in filedirs:
        fdict = load_pickle(filedir, verbose=0)
        lcobj_name = fdict['lcobj_name']
        xes = []
        for b in band_names:
            ov = fdict['error_bdict'][b]
            v = [x for x in ov if x>0]
            results_dict[method][f'{b}-fit-error'] += v
            results_dict[method][f'{b}-null-fits'] += len(v)
            results_dict[method][f'{b}-n'] += len(ov)
            results_dict[method][f'mb-fit-error'] += v
            results_dict[method][f'mb-null-fits'] += len(v)
            results_dict[method][f'mb-n'] += len(ov)
            if len(v)>0:
                xe = XError(v, 0)
                rank_bdict[method][b].add(lcobj_name, xe.mean)
                xes.append(xe)
                
        if len(xes)>0:
            rank[method].add(lcobj_name, np.mean([xe.mean for xe in xes]))
            
for method in methods:
    results_dict[method]['mb-fit-error'] = XError(results_dict[method]['mb-fit-error'], 0)
    results_dict[method]['mb-null-fits'] = (1-results_dict[method]['mb-null-fits']/results_dict[method].pop('mb-n'))*100
    for b in band_names:
        results_dict[method][f'{b}-fit-error'] = XError(results_dict[method][f'{b}-fit-error'], 0)
        results_dict[method][f'{b}-null-fits'] = (1-results_dict[method][f'{b}-null-fits']/results_dict[method].pop(f'{b}-n'))*100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
for method in methods:
    print(method, results_dict[method])

curve_fit {'mb-fit-error': XError(1.97+-9.29, dim=0), 'mb-null-fits': 9.185606060606055, 'g-fit-error': XError(1.31+-3.43, dim=0), 'g-null-fits': 13.139204545454541, 'r-fit-error': XError(2.57+-12.40, dim=0), 'r-null-fits': 5.23200757575758}
mcmc {'mb-fit-error': XError(1.67+-1.41, dim=0), 'mb-null-fits': 7.14285714285714, 'g-fit-error': XError(1.85+-1.55, dim=0), 'g-null-fits': 0.0, 'r-fit-error': XError(1.47+-1.19, dim=0), 'r-null-fits': 14.28571428571429}
